### User based Recommendation (Collaborative filtering)

-  if user A  and user B share similar profile then meal liked by User B but  not rated by A are recommended to A
- User-based: measure the similarity between target users and other users
- Collaborative filtering systems make recommendations based on historic users’ preference for items (clicked, watched, purchased, liked, rated, etc.).

### Algorithm - K_Nearest-neighbor

In [66]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv('user_dataset.csv')
df.head(3)

,Unnamed: 0,User_Id,Name,catagory,description,Veg_Non,Review,Nutrient,Disease,Diet,Price
0,0,User_54,summer squash salad,salad,"white balsamic vinegar, lemon juice, lemon rin...",veg,3,fiber,goitre obesity hypertension diabeties,dash_diet high_fiber_diet ketogenic_diet alka...,275
1,1,User_65,chicken minced salad,salad,"olive oil, chicken mince, garlic (minced), oni...",non-veg,6,fiber,goitre rickets obesity hypertension kidney_di...,high_fiber_diet ketogenic_diet hormone_diet l...,295
2,2,User_78,sweet chilli almonds,chilli,"almonds whole, egg white, curry leaves, salt, ...",veg,6,vitamin_a,heart_disease scurvy hypertension kidney_dise...,dash_diet high_fiber_diet paleo_diet alkaline...,245


In [71]:
class Recommender:
    
    def __init__(self):
        self.df = pd.read_csv('user_dataset.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(self.df.loc[i])
                
#         df_results = df_result
        df_results = df_results.drop_duplicates(subset=['Name'])
        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input])
        users = list(similar_users.User_Id)
        results = self.df[self.df.User_Id.isin(users)]
        results = results[results['User_Id']!=user_id]
        results = results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results
        
        
        
        
        

### for Example user A =='User_71' Profile says :

In [72]:
profile_A = ['calcium','iron','diabeties','gluten_free_diet']
user_id = 'User_71' # asumming for example

- finding similar users

In [73]:
ob = Recommender()
data = ob.get_features()
total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
for i in profile_A:
    d[i] = 1

final_input = list(d.values())
similar_users = ob.k_neighbor([final_input]) # pass 2d array []
similar_users

,Unnamed: 0,User_Id,Name,catagory,description,Veg_Non,Review,Nutrient,Disease,Diet,Price
0,25,User_50,almond pearls,almond,"toasted almonds, blueberries, oats, corn flake...",veg,2,protien,hypertension,gluten_free_diet,335
1,310,User_42,veg summer rolls,rice,"rice paper sheets, iceberg lettuce, carrot, be...",veg,4,carbohydrates,diabeties,high_protien_diet,665
2,217,User_52,corn pulao,rice,"basmati rice, american corn kernels, olive oil...",veg,2,iron,goitre diabeties,high_fiber_diet low_fat_diet gluten_free_diet...,245
3,14,User_92,almond and cranberry poha,poha,"almond flakes, onion, poha, cranberries (froze...",veg,9,iron,hypertension,gluten_free_diet low_fat_diet vegan_diet high...,510


- recommending meals that are new to user A and reviwed by above users

In [74]:
recommendations = ob.user_based(profile_A,user_id) 
recommendations

,Name,Nutrient,Veg_Non,Price,Review,Diet,Disease,description
0,almond and cranberry poha,iron,veg,510,9,gluten_free_diet low_fat_diet vegan_diet high...,hypertension,"almond flakes, onion, poha, cranberries (froze..."
1,grilled almond barfi (sugar free),calcium,veg,650,5,ketogenic_diet low_fat_diet gluten_free_diet ...,goitre hypertension,"khoya, sweetener (optional), almonds (crushed)"
2,almond pearls,protien,veg,335,2,gluten_free_diet,hypertension,"toasted almonds, blueberries, oats, corn flake..."
3,beetroot modak,selenium,veg,695,7,low_fat_diet vegan_diet ketogenic_diet,hypertension,"gram flour (besan), semolina (rava/sooji), bee..."
4,buldak (hot and spicy chicken),magnesium,non-veg,695,8,high_fiber_diet high_protien_diet,obesity,"chicken drum sticks/ chicken breast, soy sauce..."
5,meat lovers pizza,potassium,non-veg,450,9,dash_diet high_fiber_diet ketogenic_diet low_...,goitre obesity cancer kidney_disease anemia p...,"बाजरे का आटा, साबूदाने का आटा, सोया मिल्क, लहस..."
6,malabari fish curry,vitamin_a,non-veg,460,10,dash_diet high_fiber_diet ketogenic_diet alka...,goitre heart_disease rickets obesity cancer d...,"sear fish, coconut, ginger, pureed tamarind, s..."
7,corn pulao,iron,veg,245,2,high_fiber_diet low_fat_diet gluten_free_diet...,goitre diabeties,"basmati rice, american corn kernels, olive oil..."
8,steam bunny chicken bao,iron,non-veg,520,10,high_fiber_diet hormone_diet low_fat_diet low...,rickets anemia kidney_disease goitre pregnancy,"buns, all purpose white flour, dry yeast, suga..."
9,lemon poppy seed cake,potassium,veg,235,1,high_fiber_diet low_fat_diet ketogenic_diet h...,scurvy,"plain flour, baking powder, salt, castor sugar..."
